# Logistic growth

In this example, we are going to perform inference for the logistic growth model commonly used in population biology to describe resource-limited growth. The population density of a given bacteria is supposed to follow:

\begin{equation}
\frac{d y}{d t} = r y (1 - \frac{y}{k})
\end{equation}

where we assume $y(0)>0$; $r > 0$ is the initial (exponential) growth rate when resources are abundant; $\kappa > 0$ is the carrying capacity denoting the steady-state population density.

## 1. 
Using Scipy's integrator, write a function which numerically solves the logistic equation. Plot the solution for $y(0)=5, r=0.2, \kappa=20$ from $t=0$ to $t=40$.

## 2.
Assume that the observed data differs from the logistic model solution according to the measurement equation:

\begin{equation}
\tilde y(t) \sim \mathcal{N}(y(t), \sigma),
\end{equation}

where $\sigma >0$ controls the measurement variability. Generate 'observed' data at $t= 0, 5, 10, 15, 20, 25, 30, 35, 40$ assuming $y(0)=5, r=0.2, \kappa=20, \sigma=2$. Plot these data overlaid on top of the true solution.

## 3.
The likelihood for this model is given by:

\begin{equation}
L = \prod_{i=1}^{N} \frac{1}{\sqrt{2\pi\sigma^2}} \exp(-\frac{(\tilde y(t_i) - y(t_i))^2}{2\sigma^2})
\end{equation}

where $N$ is the number of observations and $y(t) = y(t|r,\kappa, y(0))$ is the ODE solution at time $t$. Write a function which calculates the log-likelihood for given $(r, \kappa, y(0), \sigma)$.

## 4.
Using the simulated data you previously generated, plot the log-likelihood as a function of $r$ holding all other parameters at their true values.

## 5.
Plot the likelihood (not the log-likelihood as its contours are harder to distinguish) contour surface for $(r,k)$ holding all other parameters at their true values.

## 6.

Assume we have priors on parameters: $r\sim \mathcal{N}(0.2, 0.02), \kappa\sim \mathcal{N}(20, 4)$, and we fix $y(0)=5$. Generate 100 prior predictive simulations of the ODE solution $y$ and plot these. Remember, a single prior predictive simulation is obtained by sampling each parameter from its prior and (in this case) solving the ODE for this parameter set.

## 7.
Now also allow $y(0)\sim \mathcal{N}(5, 1)$. How does the prior predictive distribution change? 

## 8.
We are now going to write a random walk Metropolis sampler. The first step is to write a method which takes as input $(r,\kappa, \sigma)$ and proposes new values using univariate normal distributions centered at the current values. So, for example,

\begin{equation}
r'\sim\mathcal{N}(r,\tau_r)
\end{equation}

where $\tau_r$ is a hyperparameter of the method. Write such a function.

## 9.
Assume priors: $r\sim \mathcal{N}(0.2, 0.02), \kappa\sim \mathcal{N}(20, 4), \sigma \sim \mathcal{N}(2, 0.2)$ and fix $y(0)=5$. Write a function that accepts as input $(r,\kappa, \sigma)$ and outputs the log-prior density $\log{\text -}p(r,\kappa, \sigma)$.

## 10.
 Write a function which calculates the unnormalised log-posterior (i.e. the sum of the log-prior and log-likelihood), $\text{log-}p(r,\kappa,\sigma|\text{data})$, for a given parameter set.

## 11.
The next step is to write a function called 'step_accept' which takes as input $(r,\kappa, \sigma)$, proposes new values $(r',\kappa', \sigma')$. Then calculates the ratio:

\begin{equation}
t = \exp(\text{log-}p(r',\kappa', \sigma'|\text{data}) - \text{log-}p(r,\kappa, \sigma|\text{data}))
\end{equation}

Then generates $u\sim U(0,1)$ and does the following: if $t \geq u$, return $(r',\kappa',\sigma')$; else return $(r,\kappa,\sigma)$.

## 12.
Write a function which iterates 'step_accept' generating a chain of MCMC samples of $(r,\kappa,\sigma)$. Initialise $(r,\kappa,\sigma)$ using samples from the prior.

## 13.
Assuming step sizes of $(\tau_r=0.01,\tau_k=1, \tau_{\sigma}=0.1)$, generate an MCMC sample of 1000 draws. Visualise the sampled values of $r$ over time.

## 14.
Plot the pairwise distribution of $(r,\kappa)$. How do the sampled values compare to the true parameters?

## 15.
Modify your MCMC routine to use the following half-normal distributions to sample initial points for your chains. Run four independent chains for 1000 iterations each and plot the $\kappa$ samples over time. How long does it appear your chains be run until they reach a stationary distribution?

$r \sim \text{half-N}(0.2, 0.1)$
$\kappa \sim \text{half-N}(0, 20, 10)$
$\sigma \sim \text{half-N}(2, 1)$

Note, to do so, you can use the following function:

<code>
def truncated_normal(mu, sd):
    myclip_a = 0
    myclip_b = 1000000
    my_mean = mu
    my_std = sd

    a, b = (myclip_a - my_mean) / my_std, (myclip_b - my_mean) / my_std
    r = scipy.stats.truncnorm.rvs(a, b, loc=my_mean, scale=my_std, size=1)
    return r[0]
</code>

## 16.
Using a random subset of 100 samples from all four chains taken from after they appear to have reached the stationary distribution, draw from the posterior predictive distribution of the logistic equation solution and overlay the observations.